In [ ]:
import bz2
import subprocess
import re
import xml.sax

In [ ]:
pip install 'setuptools>=36.2.1'

In [ ]:
pip install wikitextparser

     |████████████████████████████████| 71kB 4.7MB/s 


In [ ]:
import wikitextparser as wtp
from wikitextparser import remove_markup

In [ ]:
pip install mwparserfromhell

     |████████████████████████████████| 143kB 10.2MB/s 
  Created wheel for mwparserfromhell: filename=mwparserfromhell-0.5.4-cp36-cp36m-linux_x86_64.whl size=183776 sha256=c41ba646718b42ee884be52e3725376df8732957bd3cb7424d55dbfdc8e1c1be
  Stored in directory: /root/.cache/pip/wheels/2a/76/d5/7088b941df3b362c45dd7912dd05314bc034751ec9cbca9a75
Successfully built mwparserfromhell


In [ ]:
import mwparserfromhell

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))
#src: https://towardsdatascience.com/wikipedia-data-science-working-with-the-worlds-largest-encyclopedia-c08efbac5f5c

In [ ]:
# Object for handling xml
handler = WikiXmlHandler()# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)# Iteratively process file
for line in subprocess.Popen(['bzcat'], 
                              stdin = open("/content/drive/My Drive/Colab Notebooks/etwiki-latest-pages-articles.xml.bz2"), 
                              stdout = subprocess.PIPE).stdout:
    parser.feed(line)
#src: https://towardsdatascience.com/wikipedia-data-science-working-with-the-worlds-largest-encyclopedia-c08efbac5f5c


In [ ]:
all_pages = dict(handler._pages)
disambiguation_pages_texts = []
for el in handler._pages:
  is_disam_page = '{{täpsustus}}' in el[1] or '{{täpsustuslehekülg}}' in el[1] or '{{Täpsustus}}' in el[1] or '{{Täpsustuslehekülg}}' in el[1]
  if (is_disam_page) and el[0] != 'Vikipeedia:Vormistusreeglid':
    disambiguation_pages_texts.append(el)

In [ ]:
#make list of (disambiguation page, senses)-s
disambiguation_pages_senses = []
for el in disambiguation_pages_texts:
  senses = []
  v = el[1].split('Vaata ka')[0].split('*')
  for s in v:
    links = re.findall("\[\[(.*?)\]\]",s)
    if links != []:
      senses.append(links[0].split('|')[0])
  disambiguation_pages_senses.append((el[0],senses))

In [ ]:
# EstNLTK 1.4 kasutamiseks Colabis tuleb...
# 1. Laadida alla ja installeerida Miniconda
!wget https://repo.continuum.io/miniconda/Miniconda3-3.5.5-Linux-x86_64.sh
!chmod +x Miniconda3-3.5.5-Linux-x86_64.sh
!bash ./Miniconda3-3.5.5-Linux-x86_64.sh -b -f -p /usr/local
#!conda info
# 2. Installeerida pythoni sobiv versioon
!conda install -q --yes python=3.5
#!conda info
# 3. Lisada tee
import sys
sys.path.append('/usr/local/lib/python3.5/site-packages/')
# 4. Installeerida NLTK
!conda install -q -y nltk
# 5. Installeerida EstNLTK ja parandada mõned installeerimisel tekkinud vead
!conda install -q --yes -c estnltk -c conda-forge estnltk=1.4.1
!cp /usr/local/lib/python3.5/site-packages/estnltk/vabamorf/_vabamorf.cpython-35m-x86_64-linux-gnu.so /usr/local/lib/python3.5/site-packages/estnltk/vabamorf/_vabamorf.so
!cp /usr/local/lib/python3.5/site-packages/pycrfsuite/_pycrfsuite.cpython-35m-x86_64-linux-gnu.so /usr/local/lib/python3.5/site-packages/pycrfsuite/_pycrfsuite.so

--2020-08-21 12:28:37--  https://repo.continuum.io/miniconda/Miniconda3-3.5.5-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-3.5.5-Linux-x86_64.sh [following]
--2020-08-21 12:28:37--  https://repo.anaconda.com/miniconda/Miniconda3-3.5.5-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31457606 (30M) [application/x-sh]
Saving to: ‘Miniconda3-3.5.5-Linux-x86_64.sh’

Miniconda3-3.5.5-Li 100%[===================>]  30.00M   141MB/s    in 0.2s    

2020-08-21 12:28:38 (141 MB/s) - ‘Miniconda3-3.5.5-Linux-x86_

In [ ]:
import estnltk
from estnltk import Text

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'pycrfsuite._pycrfsuite' does not match runtime version 3.6
  return f(*args, **kwds)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#removing spaces so that parser can recognise elements
def trim_unnessessary_spaces(text):
  splitted = text.split("\n")
  for i, item in enumerate(splitted):
    splitted[i] = item.lstrip()
    splitted[i] = re.sub('< ','<',splitted[i])
    splitted[i] = re.sub(' >','>',splitted[i])
    splitted[i] = re.sub('& nbsp','&nbsp',splitted[i])
    splitted[i] = re.sub('& ndash','&ndash',splitted[i])     
  text = '\n'.join(splitted)
  return text
#function for cleaning text from markup and other, should return pure sentences in most cases
def search(text):

  regex = re.compile(r'==\s*Vaata ka\s*')
  text = regex.split(text)[0]
  regex = re.compile(r'==\s*Kirjandus\s*==')
  text = regex.split(text)[0]
  regex = re.compile(r'==\s*Viited\s*==')
  text = regex.split(text)[0]
  regex = re.compile(r'==\s*Välislingid\s*==')
  text = regex.split(text)[0]
  text = re.sub('\[\[.*:.*(\|pisi|thumb|px).*\]\]','',text)

  code = mwparserfromhell.parse(trim_unnessessary_spaces(text))

  for tag in code.filter_tags(recursive=False):
    if tag[0] == "'" and tag[-1]=="'":
      code.replace(tag,tag.contents)
    else:
      code.replace(tag,"")

  for argument in code.filter_arguments():
    code.replace(argument, "")

  for comment in code.filter_comments():
    code.replace(comment,"")

  for external_link in code.filter_external_links():
    code.replace(external_link,"")

  for heading in code.filter_headings():
    code.replace(heading,"")

  for html_entity in code.filter_html_entities():
    code.replace(html_entity, "")

  for template in code.filter_templates(recursive=False):
    code.replace(template,"")

  return remove_markup(str(code))

In [ ]:
def ok(lsnc,lword):    #function used for checking if sentence contains word
  parts_o_word = len(lword)
  if parts_o_word == 1:
    return (lword[0] in lsnc) #generally checks if element of second list is in first list
  else:                  #if multiple els in second list, eg word consists of 2 words, check if they are next to each other in 1st
    for i,el in enumerate(lsnc):
      if l1snc[i:i+parts_o_word] == lword:
        return True
  return False

In [ ]:
#takes word, text, returns sentences containing the word
def important_sentences(word,sentences_text):
  text = Text(sentences_text)
  lst_of_snts = text.split_by('sentences')
  lst_of_sntns_chosen = []
  for el in lst_of_snts:
    if ok(el.lemmas,Text(word.split(' (')[0]).lemmas):
      lst_of_sntns_chosen.append(el['text'])                 
  return lst_of_sntns_chosen

In [ ]:
#fuction that takes sense and returns (sense, sentences containing that sense)
def A(word, sense):
  raw_text = ''
  #making it work despite different capitalisation
  for el in list(all_pages.keys()):
    if sense.lower() == el.lower():
      sense = el
      break
  #if sense is a link to some page, then we can find the text of that page
  if sense in all_pages.keys():
    raw_text = all_pages[sense]
    #is it a redirect page? then we fixate the name of real page
    if bool(re.match('#suuna\s*\[\[.*?\]\]\s*',all_pages[sense])):
      raw_text = re.findall('#suuna\s*\[\[(.*?)\]\]\s*',all_pages[sense])[0].strip().replace('_', ' ')
      #does it direct to subsenction of some page?
      if bool(re.match('.*#.*',raw_text)):
        return 0
      #if it directs to page, make sure that we have right capitalization of letters
      for el in list(all_pages.keys()):
        if raw_text.lower() == el.lower():
          raw_text = el
          break
      raw_text = all_pages[raw_text]
    else:
      if bool(re.match('#REDIRECT\s*\[\[.*?\]\]\s*',all_pages[sense])):
        raw_text = re.findall('#REDIRECT\s*\[\[(.*?)\]\]\s*',all_pages[sense])[0].lower().strip().replace('_', ' ')
        #does it direct ti subsection of some page?
        if bool(re.match('.*#.*',raw_text)):
          return 0
        #if it directs to page, make sure that we have right capitalization of letters
        for el in list(all_pages.keys()):
          if raw_text.lower() == el.lower():
            raw_text = el
            break
        raw_text = all_pages[raw_text]
      else:
        if bool(re.match('#redirect\s*\[\[.*?\]\]\s*',all_pages[sense])):
          raw_text = re.findall('#redirect\s*\[\[(.*?)\]\]\s*',all_pages[sense])[0].lower().strip().replace('_', ' ')
        #does it direct ti subsection of some page?
          if bool(re.match('.*#.*',raw_text)):
            return 0
        #if it directs to page, make sure that we have right capitalization of letters
          for el in list(all_pages.keys()):
            if raw_text.lower() == el.lower():
              raw_text = el
              break
          raw_text = all_pages[raw_text]
  else:
    return 0
  #cleaning the text from unnecessary elements
  sentences_text = search(raw_text)
  #excluding sentences that don't contain the word
  important = important_sentences(word,sentences_text)
  #if there is at least 1 sentence containing the WORD, return (sense, these sentences, number of these sentences)
  if len(important) > 0:
    length = len(important)
    return (sense, important, length)
  #if no article about that sense return 0
  return 0

In [ ]:
#function that takes a WORD and list of its senses and returns (WORD, output of function A for each sense)
def B(word, senses):
  #return tuple with 2 values
  value1 = word
  value2 = []  #will contain results of function A on each sense eg (sense, clean text, number of sentences)
  #filling up value2
  for sense in senses:
    sense_and_sntnces = A(word, sense)
    if sense_and_sntnces != 0: #only if there was at least 2 senses with sentences about them
      value2.append(sense_and_sntnces)
  #accept only tuples where word has at least 2 senses which have sentences containing the word
  if len(value2) < 2:
    return 0
  return (value1,value2)

In [ ]:
all_pages['Y'] = re.sub("'''Y''' on \[\[Saksamaa\]\]l '''''\[\[Bundeswehr\]\]'' < nowiki > 'i < /nowiki > ''' \[\[auto\]\]de tähis.","",all_pages['Y'])

In [ ]:
#filtering out relevant data
nr_of_words = 0
nr_of_senses = 0
nr_of_sntnces = 0

def senses_per_word_avg(nr_of_senses,nr_of_words):
  return nr_of_senses / nr_of_words

def sntnces_per_word_avg(nr_of_sntnces,nr_of_words):
  return nr_of_sntnces / nr_of_words

def sntnces_per_sense_avg(nr_of_sntnces,nr_of_senses):
  return nr_of_sntnces / nr_of_senses

In [ ]:
with open("/content/drive/My Drive/Colab Notebooks/wiki_et.txt","w") as file1:
  for el in disambiguation_pages_senses:
    line = B(el[0],el[1])
    if line != 0:
      counter = 0
      for element in line[1]:
        counter += element[2]
      if counter != len(line[1]):
        nr_of_words += 1
        nr_of_senses += len(line[1])
        file1.write('WORD: '+line[0]+'\n')
        for i,elem in enumerate(line[1]):
          nr_of_sntnces += elem[2]
          file1.write('SENSE'+str(i)+': '+elem[0]+' - ')
          file1.write(elem[1][0].strip('\n')+'\n')
          file1.write('snc: ' + str(len(elem[1])) + '\n')
        file1.write('------------------------\n')

In [ ]:
print(nr_of_words)
print(nr_of_senses)
print(nr_of_sntnces)
print(round(senses_per_word_avg(nr_of_senses,nr_of_words),2))
print(round(sntnces_per_word_avg(nr_of_sntnces,nr_of_words),2))
print(round(sntnces_per_sense_avg(nr_of_sntnces,nr_of_senses),2))

2602
8915
42342
3.43
16.27
4.75
